# Fusion de deux jeux de données

En analyse de données, on souhaite souvent fusionner deux (ou plusieurs) jeux de données.

Cette manipulation est réalisable avec Python et le module *pandas*.

In [1]:
import pandas as pd

## Premier jeu de données

<div class="alert alert-info">

Les données expérimentales utilisées dans ce notebook proviennent d'expériences de spectrométrie de masse.  Elles ont été adaptées de l'article :  
*Quantitative Proteomics in Friedreich's Ataxia B-lymphocytes: A Valuable Approach to Decipher the Biochemical Events Responsible for Pathogenesis*  
Télot *et al*, Biochimica et Biophysica Acta (BBA) - Molecular Basis of Disease, 2018.  
DOI: [10.1016/j.bbadis.2018.01.010](https://doi.org/10.1016/j.bbadis.2018.01.010)

</div>

In [2]:
exp_df = pd.read_table("data_friedreich_ataxia.csv", sep=",", index_col="accession")

On affiche les dimensions et les premières lignes du jeu de données :

In [3]:
print("Dimensions : {} x {}".format(*exp_df.shape))
exp_df.head()

Dimensions : 2694 x 3


,abundance WT,abundance FRDA,p-value
accession,,,
P01876,4.650254e+05,5.488983e+06,0.000095
P28289,3.171518e+04,9.224471e+04,0.000013
P01871,1.596695e+06,5.501002e+04,0.000808
Q9H0E2,6.336450e+04,1.354001e+05,0.000021
P01857,8.030646e+05,1.104635e+05,0.000196


Ce jeu de données contient, pour 2694 protéines :

- l'abondance (moyenne et normalisée) mesurée par spectrométrie de masse provenant de cellules saines (condition WT) ;
- l'abondance (moyenne et normalisée) mesurée par spectrométrie de masse provenant de cellules de patients (condition FRDA) ;
- les valeurs de p-value associées.

Ces informations sont très intéressantes mais elles ne nous fournissent aucun détail sur les protéines étudiées. Nous savons que ce sont des protéines humaines, mais quel est le nom de la protéine P01876 ? P28289 ?

## Second jeu de données

Heureusement, UniProt est la base de données de référence pour les protéines. On peut télécharger, depuis UniProt, des informations sur toutes les protéines du protéome humain :

https://www.uniprot.org/uniprot/?query=reviewed:yes%20taxonomy:9606

Le jeu de données est déjà présent dans le répertoire du notebook. Mais si vous souhaitez le télécharger, ouvrez le lien web ci-dessus, cliquez sur le bouton *Download*, dans *Format* sélectionnez *Tab separated*, cochez *Uncompressed* et enfin cliquez sur le bouton *Go*. Enregistrez ce fichier avec le nom *uniprot_human.tsv*.

On ouvre ce second jeu de données avec *Pandas* :

In [4]:
uniprot_df = pd.read_table("uniprot_human.tsv", sep="\t", index_col="Entry")

On affiche les dimensions et les premières lignes du jeu de données :

In [5]:
print("Dimensions : {} x {}".format(*uniprot_df.shape))
uniprot_df.head()

Dimensions : 20366 x 6


,Entry name,Status,Protein names,Gene names,Organism,Length
Entry,,,,,,
Q8NF67,A2012_HUMAN,reviewed,Putative ankyrin repeat domain-containing prot...,ANKRD20A12P,Homo sapiens (Human),263
Q9NPB9,ACKR4_HUMAN,reviewed,Atypical chemokine receptor 4 (C-C chemokine r...,ACKR4 CCBP2 CCR11 CCRL1 VSHK1,Homo sapiens (Human),350
P31937,3HIDH_HUMAN,reviewed,"3-hydroxyisobutyrate dehydrogenase, mitochondr...",HIBADH,Homo sapiens (Human),336
P61981,1433G_HUMAN,reviewed,14-3-3 protein gamma (Protein kinase C inhibit...,YWHAG,Homo sapiens (Human),247
O94805,ACL6B_HUMAN,reviewed,Actin-like protein 6B (53 kDa BRG1-associated ...,ACTL6B ACTL6 BAF53B,Homo sapiens (Human),426


Pour chacune de ces 20366 protéines, nous avons :

- son nom dans UniProt (`Entry name`), 
- son statut (`Status`, ici elles sont toutes *reviewed*), 
- ses noms habituels (`Protein names`), 
- les noms des gènes associés (`Gene names`), 
- l'organisme (`Organism`, ici ce sont toutes des protéines humaines) 
- et sa taille (`Length`, nombre d'acides aminés).

## Fusion des jeux de données

Nous souhaitons maintenant regrouper dans un seul jeu de données, les données expérimentales mais aussi les noms et les tailles des protéines.

Nous allons pour cela utiliser la méthode `.merge()` de *Pandas* :

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html

Cette méthode est un peu plus puissante que la méthode `.concat()` présentée dans le cours en ligne.

In [6]:
new_df = pd.merge(exp_df, uniprot_df[["Protein names", "Length"]], 
                  how="left", 
                  left_index=True, right_index=True)

L'ordre des dataframes à fusionner est important pour la suite : d'abord `exp_df` puis `uniprot_df`. Comme nous ne souhaitons que les noms et la taille des protéines, nous réduisons d'emblée le second jeu de données aux colonnes d'intérêt (`uniprot_df[["Protein names", "Length"]]`).

Le paramètre `how="left"` précise que nous conservons toutes les protéines du jeu de données de gauche, c'est-à-dire `exp_df`. Pour obtenir l'intersection des deux jeux de données, nous aurions utilisé le paramètre `how="inner"`, c'est d'ailleurs ce que fait par défaut la méthode `.concat()`.

Les paramètres `left_index=True` et `right_index=True` précisent que les colonnes de référence pour faire cette jointure sont les index des 2 dataframes, qui contiennent ici les identifiants des protéines.

On affiche maintenant les dimensions et les premières lignes du nouveau jeu de données :


In [7]:
print("Dimensions : {} x {}".format(*new_df.shape))
new_df.head()

Dimensions : 2694 x 5


,abundance WT,abundance FRDA,p-value,Protein names,Length
accession,,,,,
P01876,4.650254e+05,5.488983e+06,0.000095,Immunoglobulin heavy constant alpha 1 (Ig alph...,353.0
P28289,3.171518e+04,9.224471e+04,0.000013,Tropomodulin-1 (Erythrocyte tropomodulin) (E-T...,359.0
P01871,1.596695e+06,5.501002e+04,0.000808,Immunoglobulin heavy constant mu (Ig mu chain ...,453.0
Q9H0E2,6.336450e+04,1.354001e+05,0.000021,Toll-interacting protein,274.0
P01857,8.030646e+05,1.104635e+05,0.000196,Immunoglobulin heavy constant gamma 1 (Ig gamm...,330.0


Le nouveau jeu de données possède exactement le même nombre de lignes que le jeu de données expérimentales. C'est ce que nous souhaitions.

Pour rendre l'affichage du dataframe un peu plus lisible, on peut faire en sorte que la largeur des colonnes s'adapte aux données et à largeur de la page.

In [8]:
pd.set_option('max_colwidth', 0)
new_df.head()

,abundance WT,abundance FRDA,p-value,Protein names,Length
accession,,,,,
P01876,4.650254e+05,5.488983e+06,0.000095,Immunoglobulin heavy constant alpha 1 (Ig alpha-1 chain C region) (Ig alpha-1 chain C region BUR) (Ig alpha-1 chain C region TRO),353.0
P28289,3.171518e+04,9.224471e+04,0.000013,Tropomodulin-1 (Erythrocyte tropomodulin) (E-Tmod),359.0
P01871,1.596695e+06,5.501002e+04,0.000808,Immunoglobulin heavy constant mu (Ig mu chain C region) (Ig mu chain C region BOT) (Ig mu chain C region GAL) (Ig mu chain C region OU),453.0
Q9H0E2,6.336450e+04,1.354001e+05,0.000021,Toll-interacting protein,274.0
P01857,8.030646e+05,1.104635e+05,0.000196,Immunoglobulin heavy constant gamma 1 (Ig gamma-1 chain C region) (Ig gamma-1 chain C region EU) (Ig gamma-1 chain C region KOL) (Ig gamma-1 chain C region NIE),330.0


## Recherche par nom
On peut sélectionner un ensemble de protéines par leurs noms complets. 

Par exemple, toutes les *kinases* (et les protéines associées aux *kinases*).

In [17]:
new_df[ new_df["Protein names"].str.contains("kinase", na=False) ]

,abundance WT,abundance FRDA,p-value,Protein names,Length
accession,,,,,
P43405,1.333537e+05,4.491510e+04,0.000062,Tyrosine-protein kinase SYK (EC 2.7.10.2) (Spleen tyrosine kinase) (p72-Syk),635.0
P24723,1.585143e+04,3.629752e+04,0.000242,Protein kinase C eta type (EC 2.7.11.13) (PKC-L) (nPKC-eta),683.0
P05771,2.240738e+05,5.780073e+05,0.000439,Protein kinase C beta type (PKC-B) (PKC-beta) (EC 2.7.11.13),671.0
Q3LXA3,4.693770e+05,6.947913e+05,0.002145,Triokinase/FMN cyclase (Bifunctional ATP-dependent dihydroxyacetone kinase/FAD-AMP lyase (cyclizing)) [Includes: ATP-dependent dihydroxyacetone kinase (DHA kinase) (EC 2.7.1.28) (EC 2.7.1.29) (Glycerone kinase) (Triokinase) (Triose kinase); FAD-AMP lyase (cyclizing) (EC 4.6.1.15) (FAD-AMP lyase (cyclic FMN forming)) (FMN cyclase)],575.0
Q9UHR4,3.589128e+04,1.508549e+05,0.000169,Brain-specific angiogenesis inhibitor 1-associated protein 2-like protein 1 (BAI1-associated protein 2-like protein 1) (Insulin receptor tyrosine kinase substrate),511.0
...,...,...,...,...,...
Q92616,1.592506e+06,1.577694e+06,0.966668,eIF-2-alpha kinase activator GCN1 (GCN1 eIF-2-alpha kinase activator homolog) (GCN1-like protein 1) (General control of amino-acid synthesis 1-like protein 1) (Translational activator GCN1) (HsGCN1),2671.0
P54619,4.349848e+05,4.056026e+05,0.813253,5'-AMP-activated protein kinase subunit gamma-1 (AMPK gamma1) (AMPK subunit gamma-1) (AMPKg),331.0
Q9NVE7,3.491882e+04,3.580834e+04,0.894916,4'-phosphopantetheine phosphatase (EC 3.1.3.-) (Inactive pantothenic acid kinase 4) (hPanK4),773.0


203 protéines associées aux kinases. C'est peut être intéressant (ou pas).

Et si on souhaite trier cette même sélection par taille de protéine décroissante, on utilise la méthode `.sort_values()`

In [18]:
new_df[ new_df["Protein names"].str.contains("kinase", na=False) ].sort_values(by=["Length"], ascending=False)

,abundance WT,abundance FRDA,p-value,Protein names,Length
accession,,,,,
P78527,1.381797e+06,1.361247e+06,0.854557,DNA-dependent protein kinase catalytic subunit (DNA-PK catalytic subunit) (DNA-PKcs) (EC 2.7.11.1) (DNPK1) (p460),4128.0
Q96Q15,6.928457e+04,8.189093e+04,0.302246,Serine/threonine-protein kinase SMG1 (SMG-1) (hSMG-1) (EC 2.7.11.1) (61E3.4) (Lambda/iota protein kinase C-interacting protein) (Lambda-interacting protein),3661.0
Q13315,6.561895e+04,7.758885e+04,0.644410,Serine-protein kinase ATM (EC 2.7.11.1) (Ataxia telangiectasia mutated) (A-T mutated),3056.0
Q12802,7.510475e+04,1.220768e+05,0.020962,A-kinase anchor protein 13 (AKAP-13) (AKAP-Lbc) (Breast cancer nuclear receptor-binding auxiliary protein) (Guanine nucleotide exchange factor Lbc) (Human thyroid-anchoring protein 31) (Lymphoid blast crisis oncogene) (LBC oncogene) (Non-oncogenic Rho GTPase-specific GTP exchange factor) (Protein kinase A-anchoring protein 13) (PRKA13) (p47),2813.0
Q92616,1.592506e+06,1.577694e+06,0.966668,eIF-2-alpha kinase activator GCN1 (GCN1 eIF-2-alpha kinase activator homolog) (GCN1-like protein 1) (General control of amino-acid synthesis 1-like protein 1) (Translational activator GCN1) (HsGCN1),2671.0
...,...,...,...,...,...
P63208,1.322223e+06,1.140823e+06,0.062391,S-phase kinase-associated protein 1 (Cyclin-A/CDK2-associated protein p19) (p19A) (Organ of Corti protein 2) (OCP-2) (Organ of Corti protein II) (OCP-II) (RNA polymerase II elongation factor-like protein) (SIII) (Transcription elongation factor B polypeptide 1-like) (p19skp1),163.0
P22392,8.694864e+06,8.338635e+06,0.801325,Nucleoside diphosphate kinase B (NDK B) (NDP kinase B) (EC 2.7.4.6) (C-myc purine-binding transcription factor PUF) (Histidine protein kinase NDKB) (EC 2.7.13.3) (nm23-H2),152.0
P15531,8.349134e+06,9.692829e+06,0.436740,Nucleoside diphosphate kinase A (NDK A) (NDP kinase A) (EC 2.7.4.6) (Granzyme A-activated DNase) (GAAD) (Metastasis inhibition factor nm23) (NM23-H1) (Tumor metastatic process-associated protein),152.0


## Recherche des valeurs manquantes

Notre jeu de données expérimentales contient 2694 protéines. Le jeu de données d'UniProt contient 20366 protéines, soit toutes les protéines humaines.

Donc a priori, toutes les protéines de notre jeu de données expérimentales ont du récupérer un nom et une longueur.

Nous pouvons le vérifier :

In [10]:
new_df[ new_df.isna().any(axis=1) ]

,abundance WT,abundance FRDA,p-value,Protein names,Length
accession,,,,,
P10319,691381.466516,739355.814321,0.793366,NaN,NaN


Zut ! Des informations sont manquantes pour une protéine.
Pourtant, le jeu de données d'UniProt était complet...

Si on cherche sur [UniProt](https://www.uniprot.org/) cette protéine (*P10319*), on retombe sur la protéine [P01889](https://www.uniprot.org/uniprot/P01889). UniProt est une base de données mise à jour en permanence et certains identifiants sont remplacés par d'autres. L'identifiant *P10319* a été remplacé par l'identifiant *P01889* qui, lui, est bien présent dans notre jeu de données provenant d'UniProt :

In [11]:
uniprot_df.loc["P01889"]

Entry name       HLAB_HUMAN                                                                               
Status           reviewed                                                                                 
Protein names    HLA class I histocompatibility antigen, B alpha chain (Human leukocyte antigen B) (HLA-B)
Gene names       HLA-B HLAB                                                                               
Organism         Homo sapiens (Human)                                                                     
Length           362                                                                                      
Name: P01889, dtype: object

Dans ce cas, on peut manuellement remplacer l'identifiant obsolète par le nouveau :

In [19]:
exp_df = exp_df.rename(index={"P10319":"P01889"})

Puis refaire les manipulations précédentes :

In [20]:
new_df = pd.merge(exp_df, uniprot_df[["Protein names", "Length"]], 
                  how="left", 
                  left_index=True, right_index=True)
new_df.loc["P01889"]

abundance WT      691381                                                                                   
abundance FRDA    739356                                                                                   
p-value           0.793366                                                                                 
Protein names     HLA class I histocompatibility antigen, B alpha chain (Human leukocyte antigen B) (HLA-B)
Length            362                                                                                      
Name: P01889, dtype: object

Ouf, toutes les protéines quantifiées par spectrométrie de masse sont désormais annotées.

<div class="alert alert-block alert-warning">

Si nous avions utilisé la méthode `.concat()` ou la méthode `.merge()` avec le paramètre `how="inner"`, nous aurions obtenu une intersection des deux jeux de données. La protéine P10319 n'étant pas présente dans le jeu de données provenant d'UniProt, elle aurait été éliminée et le dataframe final n'aurait eu que 2693 lignes. 

</div>
